In [303]:
import pandas as pd
import numpy as np
import os  
import torch
from torch import nn
from sklearn.model_selection import train_test_split


In [304]:
def filepath(f):
    d = os.path.join(os.path.dirname(os.getcwd()), 'processed_data', f)
    return d

In [305]:
#Import training and test datasets
train_data=pd.read_csv(filepath("final_training_set.csv"))
test_data=pd.read_csv(filepath("final_test_set.csv"))

In [306]:
train_data

,Provider,PotentialFraud,ClaimID,InscClaimAmtReimbursed,DeductibleAmtPaid,inpatient,Gender,Race,RenalDiseaseIndicator,ChronicCond_Alzheimer,...,procedure_9,procedure_10,procedure_11,procedure_12,procedure_13,procedure_14,procedure_15,procedure_16,procedure_17,procedure_18
0,PRV51001,0,CLM733300,20,0.0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,PRV51001,0,CLM372475,700,0.0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,PRV51001,0,CLM748221,900,0.0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PRV51001,0,CLM272936,500,0.0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,PRV51001,0,CLM58316,36000,1068.0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,PRV57763,0,CLM582682,1100,40.0,0,2,2,0,1,...,0,0,0,0,0,0,0,0,0,0
558207,PRV57763,0,CLM553988,200,0.0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
558208,PRV57763,0,CLM610854,80,0.0,0,2,1,1,0,...,0,0,0,0,0,0,0,0,0,0
558209,PRV57763,0,CLM691834,3300,0.0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [307]:
uid = train_data["ClaimID"]
train_data = train_data.drop(
    ["ClaimID"],
    axis = 1
)

In [308]:
for c in train_data.columns:
    print(c)

Provider
PotentialFraud
InscClaimAmtReimbursed
DeductibleAmtPaid
inpatient
Gender
Race
RenalDiseaseIndicator
ChronicCond_Alzheimer
ChronicCond_Heartfailure
ChronicCond_KidneyDisease
ChronicCond_Cancer
ChronicCond_ObstrPulmonary
ChronicCond_Depression
ChronicCond_Diabetes
ChronicCond_IschemicHeart
ChronicCond_Osteoporasis
ChronicCond_rheumatoidarthritis
ChronicCond_stroke
IPAnnualReimbursementAmt
IPAnnualDeductibleAmt
OPAnnualReimbursementAmt
OPAnnualDeductibleAmt
age
claim_duration
time_under_care
admitDiagInFinalDiagnosis
diagnosis_1
diagnosis_2
diagnosis_3
diagnosis_4
diagnosis_5
diagnosis_6
diagnosis_7
diagnosis_8
diagnosis_9
diagnosis_10
diagnosis_11
diagnosis_12
diagnosis_13
diagnosis_14
diagnosis_15
diagnosis_16
diagnosis_17
diagnosis_18
diagnosis_19
procedure_1
procedure_2
procedure_3
procedure_4
procedure_5
procedure_6
procedure_7
procedure_8
procedure_9
procedure_10
procedure_11
procedure_12
procedure_13
procedure_14
procedure_15
procedure_16
procedure_17
procedure_18


In [309]:
def minmax_encode(df, col):
    """
    Return dataset including the minmax encoded column and excluding the original column

    Constraints:
    - col must be a String
    - df must be a Pandas Dataframe
    - df[col] must be numeric
    """

    maxx = df[col].max()
    minx = df[col].min()
    out = list(map(lambda x: (x-minx)/(maxx-minx), df[col]))
    new_colname = col + "_minmax"
    df[new_colname] = out
    return df.drop(
        [col],
        axis = 1
    )

In [310]:
def one_hot_encode(df, col):
    """
    Returns the dataset including the one hot encoded columns and excluding the original column

    Constraints:
    - col must be a String
    - df must be a Pandas Dataframe
    - df[col] must be a Series that represents a categorical variable
    """
    ohe_cols = pd.get_dummies(df[col], prefix = col)
    output = pd.concat(
        [df, ohe_cols],
        axis = 1,
    ).drop(
        [col],
        axis = 1
    )
    return output

def frequency_encode(df, col):
    """
    Returns the dataset including the frequency encoded column and excluding the original column

    Constraints:
    - col must be a String
    - df must be a Pandas Dataframe
    - df[col] must be a Series that represents a categorical variable with high cardinality
    """
    val_counts = df[col].value_counts().to_dict()
    total = len(col)
    out = []
    for x in df[col]:
        out.append(val_counts[x]/total)
    new_colname = col + '_freq'
    df[new_colname] = out
    df.drop(
        [col],
        axis = 1,
        inplace = True
    )

    return df

In [311]:
freq_encoded_cols = []
ohe_cols = []
num_cols = []
unique_threshold = 30

for col in train_data.columns:
    if train_data[col].nunique() == 1:
        print(col,"has been removed as it is constant")
        train_data.drop([col], axis=1, inplace=True)
    elif train_data[col].nunique() == 2: # Binary columns
        continue
    elif train_data[col].dtype in ['int64','float64']:
        train_data[col] = train_data[col].fillna(train_data[col].median())
        num_cols.append(col)
    elif train_data[col].nunique() > unique_threshold:
        freq_encoded_cols.append(col)
    elif 2 < train_data[col].nunique() <= unique_threshold:
        ohe_cols.append(col)
        


procedure_1 has been removed as it is constant
procedure_2 has been removed as it is constant
procedure_3 has been removed as it is constant


In [312]:
for col in train_data.columns:
    if col in num_cols:
        train_data = minmax_encode(train_data, col)
    elif col in ohe_cols:
        train_data = one_hot_encode(train_data, col)
    elif col in freq_encoded_cols:
        try:
            train_data = frequency_encode(train_data, col)
        except:
            print(col)
    elif train_data[col].nunique() == 1:
        train_data.drop(
            [col],
            axis = 1,
            inplace = True
        )
    else:
        train_data[col] = train_data[col].astype('bool')

In [313]:
y = train_data["PotentialFraud"]
train_data.drop(
    ["PotentialFraud"],
    axis = 1,
    inplace = True
)

In [314]:
# Load the data in as a Pytorch tensor
BATCH_SIZE = 64
train_tensor = torch.tensor(train_data.values.tolist())
train_dataloader = torch.utils.data.DataLoader(
    dataset = train_tensor,
    shuffle = True,
    batch_size = BATCH_SIZE
)

Variational Autoencoder Model

In [315]:
curr_col_count = len(train_data.columns)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [316]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE,self).__init__()

        self.L1 = curr_col_count
        self.L2 = curr_col_count//2
        self.L3 = curr_col_count//4

        self.mean_layer = nn.Linear(self.L3,2)
        self.var_layer = nn.Linear(self.L3,2)

    def encode(self,x):
        model = nn.Sequential(
            #nn.Dropout(0.2),
            nn.Linear(self.L1, self.L2),
            nn.LeakyReLU(0.2),
            #nn.Dropout(0.2),
            nn.Linear(self.L2, self.L3),
            nn.LeakyReLU(0.2),
        )
        latent_output = model(x)
        mean,var = self.mean_layer(latent_output), self.var_layer(latent_output)
        return mean, var
    
    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var).to(device)      
        z = mean + var*epsilon
        return z

    def decode(self, x):
        model = nn.Sequential(
            nn.Linear(2, self.L3),
            nn.LeakyReLU(0.2),
            nn.Linear(self.L3, self.L2),
            nn.LeakyReLU(0.2),
            #nn.Dropout(0.2),
            nn.Linear(self.L2, self.L1),
            nn.LeakyReLU(0.2),
            nn.Sigmoid()
        )
        return model(x)
        
    def forward(self,x):
        mean, var = self.encode(x)
        z = self.reparameterization(mean, var)
        x_hat = self.decode(z)
        return x_hat, mean, var


In [317]:
# Define model
model = VAE().to(device)

# Define loss function (sum of binary crossentropy and KL Divergence)
def loss_function(x,x_hat,mean,var):
    reproduction_loss= nn.functional.binary_cross_entropy(x_hat,x,reduction="sum")
    KL_Divergence = -0.5 * torch.sum(1+var-mean.pow(2) - var.exp())
    return reproduction_loss+KL_Divergence

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)



In [318]:
def train(model, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        overall_loss = 0
        for batch_idx, x in enumerate(train_dataloader):
            optimizer.zero_grad()
            x_hat, mean, var = model(x)
            loss = loss_function(x, x_hat, mean, var)
            overall_loss += loss.item()
            loss.backward()
            optimizer.step()
        print("\tEpoch", epoch + 1, "\tAverage Loss: ", overall_loss/(batch_idx*BATCH_SIZE))
    return loss



In [319]:
EPOCHS = 50
train(model, optimizer, EPOCHS)

	Epoch 1 	Average Loss:  17897609841.625122
	Epoch 2 	Average Loss:  29063455.99513741
	Epoch 3 	Average Loss:  9381604315.615726
	Epoch 4 	Average Loss:  208.87940243787926
	Epoch 5 	Average Loss:  218.05860723783337
	Epoch 6 	Average Loss:  2085.2944261859425
	Epoch 7 	Average Loss:  195.4147152169818
	Epoch 8 	Average Loss:  36.23189568019356
	Epoch 9 	Average Loss:  23.649729160705146
	Epoch 10 	Average Loss:  1530.8263686804617
	Epoch 11 	Average Loss:  71.46109692585958
	Epoch 12 	Average Loss:  124.9961467052143
	Epoch 13 	Average Loss:  77.33498806022389
	Epoch 14 	Average Loss:  70.33267534101017
	Epoch 15 	Average Loss:  21.624257386058765
	Epoch 16 	Average Loss:  132.28070886815476
	Epoch 17 	Average Loss:  43407512.01159772
	Epoch 18 	Average Loss:  1171.038876728327
	Epoch 19 	Average Loss:  99725613.40346871
	Epoch 20 	Average Loss:  19781.15306460737
	Epoch 21 	Average Loss:  68.2211427066447
	Epoch 22 	Average Loss:  110.51173852899703
	Epoch 23 	Average Loss:  364.626

tensor(178.5784, grad_fn=<AddBackward0>)